# 0. 导入库 

In [19]:
import os
import sys


import pickle

import pandas as pd
import numpy as np

import torch
import torch.nn as no
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertModel

# 1. 处理数据

In [3]:
DATA_DIR = '/mnt/HDD2/lyp/Dataset/Transformer_learn'

df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:

cols = df.columns
label_cols = list(cols[2:])
num_labels = len(label_cols)

df = df.sample(frac=1).reset_index(drop=True) #shuffle rows
df['one_hot_labels'] = list(df[label_cols].values)

labels = list(df.one_hot_labels.values)
comments = list(df.comment_text.values)

# 取小数据集测试

labels = labels[:320]
comments = comments[:320]

# 2. 构造 DataLoader

In [22]:
MAX_LEENGTH = 100

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encodings = tokenizer(comments,
                        padding='max_length',
                        truncation=True,
                        max_length = MAX_LEENGTH,
                        )
encodings.keys()

# return_tensors='pt'

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
input_ids = encodings['input_ids']
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [31]:

train_inputs, validation_inputs  = input_ids[:300], input_ids[301: 321] 
train_token_types, validation_token_types = token_type_ids[:300], token_type_ids[301: 321] 
train_masks, validation_masks  = attention_masks[:300], attention_masks[301: 321] 
train_labels, validation_labels = labels[:300], labels[301: 321] 


# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

#

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
# 3. Load Model & Set Params
